In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
dftrain = pd.read_csv('LabelEncoded.csv')

In [3]:
dftrain

,Unnamed: 0,Year,Severity,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Airport_Code,Temperature(F),Wind_Chill(F),Visibility(mi),Wind_Direction,Weather_Condition,Traffic_Signal,Sunrise_Sunset,TimeDiff
0,0,0,2,39.063148,-84.032608,0.010,35986,6679,243,33,748,36.0,33.3,10.0,15,46,1,1,30.0
1,1,0,3,39.747753,-84.205582,0.010,19445,1468,759,33,394,35.1,31.0,9.0,15,43,0,1,30.0
2,2,0,2,39.627781,-84.188354,0.010,23348,1468,759,33,979,36.0,33.3,6.0,15,43,1,0,30.0
3,3,0,3,40.100590,-82.925194,0.010,41325,6596,399,33,316,37.9,35.5,7.0,14,30,0,0,30.0
4,4,0,2,39.758274,-84.230507,0.000,26102,1468,759,33,394,34.0,31.0,7.0,21,46,0,0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243014,243014,1,2,38.765632,-111.353007,0.159,38077,5113,1032,42,1524,57.0,57.0,10.0,15,47,0,1,360.0
243015,243015,1,2,39.764370,-105.593050,0.321,19432,1634,239,4,3,46.4,37.2,10.0,22,46,0,0,360.0
243016,243016,1,4,39.919750,-75.576910,0.277,3955,6519,218,36,1149,63.0,63.0,10.0,0,6,1,1,360.0
243017,243017,1,4,36.034630,-79.054100,0.331,2251,1016,828,25,764,63.0,63.0,10.0,7,43,0,1,360.0


In [4]:
cat_cols = ['Street','City','County','State','Airport_Code','Wind_Direction','Weather_Condition','Sunrise_Sunset']

# Convert categorical columns to category type
for col in cat_cols:
    dftrain[col] = dftrain[col].astype('category')

In [5]:
for cols in dftrain.columns:
    print(cols, len(dftrain[cols].unique()))

Unnamed: 0 243019
Year 3
Severity 4
Start_Lat 134988
Start_Lng 134347
Distance(mi) 3674
Street 42361
City 6857
County 1277
State 49
Airport_Code 1616
Temperature(F) 271
Wind_Chill(F) 573
Visibility(mi) 49
Wind_Direction 23
Weather_Condition 70
Traffic_Signal 2
Sunrise_Sunset 2
TimeDiff 3822


In [6]:
df1 = dftrain.drop(columns=['Unnamed: 0'])

In [7]:
df_ = df1.drop(columns=['Severity'])

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import re as re
from collections import Counter

from tqdm.auto import tqdm
import math
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
import warnings
warnings.filterwarnings('ignore')

import time
from xgboost import XGBClassifier
%matplotlib inline
tqdm.pandas()


In [9]:
X = df1.drop(columns=['Severity'])
y = df1['Severity']
y = y-np.ones(y.shape[0])

In [10]:
y,df1['Severity']

(0         1.0
 1         2.0
 2         1.0
 3         2.0
 4         1.0
          ... 
 243014    1.0
 243015    1.0
 243016    3.0
 243017    3.0
 243018    3.0
 Name: Severity, Length: 243019, dtype: float64,
 0         2
 1         3
 2         2
 3         3
 4         2
          ..
 243014    2
 243015    2
 243016    4
 243017    4
 243018    4
 Name: Severity, Length: 243019, dtype: int64)

In [11]:
from sklearn.model_selection import train_test_split
X_train_, X_test, y_train_, y_test = train_test_split(X,y , random_state=42,test_size=0.20, shuffle=False)

In [13]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=1000,
    n_jobs=-1,
    max_depth=3,
    eta=0.2,
    colsample_bytree=0.67,
    enable_categorical=True,
    verbosity=0
)

model.fit(
    X_train_,
    y_train_,
    eval_set=[(X_train_, y_train_), (X_valid, y_valid)],
    # eval_metric="auc",
    # early_stopping_rounds=300,
    verbose=False
)


NameError: name 'X_valid' is not defined

In [ ]:
from sklearn.metrics import accuracy_score

# Predict on test set
y_pred = model.predict(X_test)

# Accuracy
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc*100, "%")


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred)
print(cm)

# Visualize
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')


In [ ]:
FOLDS = 20
SEED = 1004
xgb_models = []
xgb_oof = []
predictions = np.zeros(((y_test.shape[0]),4))
f_imp = []
xgb_valAccu=[]
xgb_TrainAccu=[]

counter = 0
# X = X_train_
# y = y_train_
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    if (fold + 1) % 5 == 0 or (fold + 1) == 1:
        print(f'{"#" * 24} Training FOLD {fold + 1} {"#" * 24}')
    
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]
    watchlist = [(X_train, y_train), (X_valid, y_valid)]

    # XGboost model and fit with GPU support
    model = XGBClassifier(n_estimators=1000, n_jobs=-1, max_depth=3, eta=0.2, colsample_bytree=0.67, tree_method='gpu_hist',enable_categorical=True,verbosity=0)
    model.fit(X_train, y_train, eval_set=watchlist, early_stopping_rounds=300,verbose=False)

    val_preds = model.predict_proba(X_valid)
    train_preds = model.predict_proba(X_train)
    val_pred = model.predict(X_valid)
#     val_accu = accuracy_score(y_valid, val_preds)
#     print(val_accu)
    val_score = roc_auc_score(y_valid, val_preds,multi_class='ovr')
    train_score = roc_auc_score(y_train, train_preds,multi_class='ovr')
    best_iter = model.best_iteration
    xgb_valAccu.append(val_score)
    xgb_TrainAccu.append(train_score)
#     idx_pred_target = np.vstack([val_idx, val_preds, y_valid]).T  # shape(len(val_idx), 3)
#     f_imp.append({i: j for i, j in zip(X.columns, model.feature_importances_)})
    print(f'{" " * 20}AUC: {val_score:.5f} {" " * 6}Best Iteration: {best_iter}')

    if val_score > 0.80 and train_score > 0.90:
        test_preds = model.predict_proba(X_test)
        predictions += test_preds
        counter += 1

if(counter!=0):
    predictions /= counter
# mean_val_auc = get_mean_auc(np.array(xgb_oof))
xgb_TestAccu = xgb_valAccu

In [ ]:
print("Mean of Training score")
print(np.mean(xgb_TrainAccu))

In [ ]:
print("Mean of Testing score")
print(np.mean(xgb_TestAccu))

In [ ]:
import pickle

# Assuming `model` is the last trained XGBoost model from your loop
# Save it as a .pkl file
with open("xgboost_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model saved successfully!")
